In [2]:
from ortools.algorithms import pywrapknapsack_solver

In [3]:
solver = pywrapknapsack_solver.KnapsackSolver(pywrapknapsack_solver.KnapsackSolver.KNAPSACK_DYNAMIC_PROGRAMMING_SOLVER,'test')


weights = [[565, 406, 194, 130, 435, 367, 230, 315, 393,125, 670, 892, 600, 293, 712, 147, 421, 255]]
capacities = [850]
values = weights[0]
solver.Init(values, weights, capacities)
computed_value = solver.Solve()

packed_items = [x for x in range(0, len(weights[0])) if solver.BestSolutionContains(x)]
packed_weights = [weights[0][i] for i in packed_items]

print("Packed items: ", packed_items)
print("Packed weights: ", packed_weights)
print("Total weight (same as total value): ", computed_value)

Packed items:  [2, 3, 6, 13]
Packed weights:  [194, 130, 230, 293]
Total weight (same as total value):  847


In [1]:
import sys
import os
import json
import matplotlib.pyplot as plt
from pprint import pprint
import numpy as np
from datetime import datetime
import pandas as pd
import seaborn as sns

sys.path.append(os.path.abspath('../'))
from data import Data

d = Data()

tracks = d.getTrackFeatures()

tracks

array([[0.458, 0.591, 5, ..., 184.913, 161187, 3],
       [0.455, 0.623, 8, ..., 182.345, 220293, 4],
       [0.742, 0.753, 1, ..., 132.064, 222727, 4],
       ...,
       [0.488, 0.275, 2, ..., 97.547, 242667, 3],
       [0.624, 0.851, 9, ..., 128.03, 185578, 4],
       [0.553, 0.891, 4, ..., 82.011, 252794, 4]], dtype=object)

## MIP

In [9]:
from ortools.linear_solver import pywraplp

# Instantiate a mixed-integer solver
solver = pywraplp.Solver('SolveTransportationProblem', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

cost = [[90, 76, 75, 70, 50, 74, 12, 68],
      [35, 85, 55, 65, 48, 101, 70, 83],
      [125, 95, 90, 105, 59, 120, 36, 73],
      [45, 110, 95, 115, 104, 83, 37, 71],
      [60, 105, 80, 75, 59, 62, 93, 88],
      [45, 65, 110, 95, 47, 31, 81, 34],
      [38, 51, 107, 41, 69, 99, 115, 48],
      [47, 85, 57, 71, 92, 77, 109, 36],
      [39, 63, 97, 49, 118, 56, 92, 61],
      [47, 101, 71, 60, 88, 109, 52, 90]]

task_sizes = [10, 7, 3, 12, 15, 4, 11, 5]

# Maximum total of task sizes for any worker
total_size_max = 15
num_workers = len(cost)
num_tasks = len(cost[1])
# Variables
x = {}

for i in range(num_workers):
    for j in range(num_tasks):
      x[i, j] = solver.IntVar(0, 1, 'x[%i,%i]' % (i, j))

  # Constraints

  # The total size of the tasks each worker takes on is at most total_size_max.

for i in range(num_workers):
    solver.Add(solver.Sum([task_sizes[j] * x[i, j] for j in range(num_tasks)]) <= total_size_max)

  # Each task is assigned to at least one worker.

for j in range(num_tasks):
    solver.Add(solver.Sum([x[i, j] for i in range(num_workers)]) >= 1)

solver.Minimize(solver.Sum([cost[i][j] * x[i,j] for i in range(num_workers)
                                                  for j in range(num_tasks)]))
sol = solver.Solve()


print('Total cost = ', solver.Objective().Value())
print()
for i in range(num_workers):
    for j in range(num_tasks):
        if x[i, j].solution_value() > 0:
            print('Worker', i,' assigned to task', j, '  Cost = ', cost[i][j])
print()
print("Time = ", solver.WallTime(), "milliseconds")

Total cost =  326.0

Worker 0  assigned to task 6   Cost =  12
Worker 1  assigned to task 0   Cost =  35
Worker 1  assigned to task 2   Cost =  55
Worker 4  assigned to task 4   Cost =  59
Worker 5  assigned to task 5   Cost =  31
Worker 5  assigned to task 7   Cost =  34
Worker 6  assigned to task 1   Cost =  51
Worker 8  assigned to task 3   Cost =  49

Time =  61 milliseconds


## Set Creation